In [9]:
from Bio import AlignIO, SeqIO, Align
import pandas as pd
import os, re, sys, shutil, subprocess
from copy import deepcopy

# IMPORTANT: EDIT THE PARAMETERS BELOW BEFORE RUNNING!
# ------------------------------------------------------------------------------------------------
# The directory where this notebook will create files and work from.
# The directory should either:
#   (a) be empty
#   (b) contain a ./input directory you pre-populated with fastas containing the genes to align (one per organism)
cwd = '/home/ecutts/Bacteroidetes/rprot-concatenations/rprot-aln-all-chlorobi/'

overwrite_input = True
# If True, preexisting files in ./input will be DELETED and replaced with files from input_directories

input_ext = '.fasta'
# Extension of inputs. Files in input_directories ending in input_ext will be copied to ./input

input_directories = ['/home/ecutts/Bacteroidetes/Refseq-blast/rprots/all/', \
                     '/home/ecutts/Bacteroidetes/MAGs-blast/rprots/edited', \
                    '/home/ecutts/Bacteroidetes/Chlorobi/rprots/']
# Directories (absolute path) containing inputs (can be left blank if ./input was pre-populated)

map_names = True
# Map sequence names to filenames, useful for mapping taxa to filenames
# If True, edit the function get_name_from_file() to work for your file naming scheme
# see below for example

genes = {} # see below, I wrote code to generate this automatically
# Dictionary mapping gene names (for files) to search terms (for text searching)
# Search terms should be specific (i.e. will only return one gene in your fastas)


# EXTRA CODE SPECIFIC TO YOUR FILES AND GENES
# ------------------------------------------------------------------------------------------------
# Populate the gene_search_terms dict with conserved rprots
for g in pd.read_table('/home/ecutts/metadata/rprots.txt', header=None)[0].tolist():
    genes[g] = ' ' + g + ' [' # "L1" can also show up in "L13", so this makes the search terms specific.
    
# get_name_from_taxa()
# change 
from ete3 import NCBITaxa
ncbi = NCBITaxa()

def get_name_from_file(file):
    if 'rprots_MAG' in file: # If file is a Shark Bay MAG
        name = file.split('rprots_')[1].split('.fasta')[0]
        return name.replace('_', ' ') # Return MAG ##
    else: # else, the file is a NCBI refseq genome with a genus, species name
        taxid = int(file.split('_')[1])
        genus_species = ncbi.translate_to_names([taxid])[0]
        return(str(taxid) + ' ' + genus_species)

In [10]:
if not os.path.exists(cwd):
    os.makedirs(cwd)
os.chdir(cwd)
if os.path.exists('./input') and overwrite_input == True:
    shutil.rmtree('./input')
os.makedirs('./input', exist_ok=True)
os.makedirs('./pre-aln', exist_ok=True) # ./gene-pre-aln will contain the pre-aligned files for each gene
os.makedirs('./aln', exist_ok=True) # ./gene-aln will contain the individual alignments for each gene

In [11]:
# Import files from input_directories ending in input_ext
preexisting = [f for f in os.listdir('./input') if f.endswith(input_ext)]
for d in input_directories:
    for f in os.listdir(d):
        if f not in preexisting and f.endswith(input_ext):
            shutil.copy2(os.path.join(d,f), './input')
            
inputs = [f for f in os.listdir('./input') if f.endswith(input_ext)] # only use files with extension input_ext
print('Number of input files: ' + str(len(inputs)))

Number of input files: 323


In [12]:
for g in genes.keys():
    search_term = genes[g]
    with open('pre-aln/' + g + '.fasta', 'w') as outfile:
        for f in inputs:
            file = './input/' + f
            if map_names == True:
                name = get_name_from_file(file)
            else:
                name = f
            with open(file, 'r') as infile:
                for record in SeqIO.parse(infile, 'fasta'):
                    if search_term in record.description:
                        record.id = name
                        record.description = ""
                        record.name = name
                        SeqIO.write(record, outfile, 'fasta')       

In [13]:
print('Checksum: # of sequences in each pre-align file')
print('If all genomes have all genes, checksum = ' + str(len(os.listdir('input'))))
print()
check = len(os.listdir('input'))
for f in os.listdir('./pre-aln'):
    checksum = 0
    with open('pre-aln/' + f, 'r') as checkfile:
        for record in SeqIO.parse(checkfile, 'fasta'):
            checksum +=1
    if checksum != check:   
        print(f + ': ' + str(checksum) + ' ***')
    else:
        print(f + ': ' + str(checksum))

Checksum: # of sequences in each pre-align file
If all genomes have all genes, checksum = 323

L1.fasta: 320 ***
L2.fasta: 320 ***
L3.fasta: 321 ***
L4.fasta: 322 ***
L5.fasta: 318 ***
L6.fasta: 319 ***
L10.fasta: 321 ***
L13.fasta: 322 ***
L14.fasta: 318 ***
L15.fasta: 319 ***
L18.fasta: 321 ***
L22.fasta: 318 ***
L23.fasta: 322 ***
L24.fasta: 318 ***
L29.fasta: 315 ***
S2.fasta: 321 ***
S3.fasta: 320 ***
S4.fasta: 321 ***
S5.fasta: 321 ***
S7.fasta: 318 ***
S8.fasta: 319 ***
S9.fasta: 322 ***
S10.fasta: 321 ***
S11.fasta: 320 ***
S12.fasta: 319 ***
S13.fasta: 319 ***
S14.fasta: 319 ***
S15.fasta: 320 ***
S17.fasta: 317 ***
S19.fasta: 321 ***


In [15]:
# This block will identify the files that have "problems" with the checksum above
# Comment out or skip if your checksum is good.
print('Missing and duplicate genes:')
print()
for file in os.listdir('./input'):
    genedict = dict.fromkeys(genes.values(), 0)
    with open('input/' + file, 'r') as readfile:
        for record in SeqIO.parse(readfile, 'fasta'):
            for gene in genedict.keys():
                if gene in record.description:
                    genedict[gene] += 1
    for gene in genedict.keys():
        if genedict[gene] != 1:
            print('search term: ' + "'" + gene + "' - "  + str(genedict[gene]) + ' in ' + file)

Missing and duplicate genes:

search term: ' L1 [' - 0 in rprots_496057_GCF_007971385.1.fasta
search term: ' L2 [' - 0 in rprots_496057_GCF_007971385.1.fasta
search term: ' L3 [' - 0 in rprots_496057_GCF_007971385.1.fasta
search term: ' L5 [' - 0 in rprots_496057_GCF_007971385.1.fasta
search term: ' S11 [' - 0 in rprots_496057_GCF_007971385.1.fasta
search term: ' S7 [' - 0 in rprots_1189621_GCF_000265075.1.fasta
search term: ' S15 [' - 0 in rprots_1189621_GCF_000265075.1.fasta
search term: ' L5 [' - 0 in rprots_2518971_GCF_004803915.1.fasta
search term: ' L6 [' - 0 in rprots_2518971_GCF_004803915.1.fasta
search term: ' L15 [' - 0 in rprots_1605367_GCF_004329705.1.fasta
search term: ' S2 [' - 0 in rprots_2049305_GCF_006974145.1.fasta
search term: ' L22 [' - 0 in rprots_1912246_GCF_008085455.1.fasta
search term: ' L2 [' - 0 in rprots_MAG_15.fasta
search term: ' L14 [' - 0 in rprots_MAG_15.fasta
search term: ' L22 [' - 0 in rprots_MAG_15.fasta
search term: ' L24 [' - 0 in rprots_MAG_15.fa

In [16]:
mafft = '/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft' # path to mafft

def align(file): # function that performs alignment and prints process info
    output = open('./aln/' + file + '.aln', 'w')
    print(
        subprocess.run([mafft, '--reorder', '--auto', './pre-aln/' + file], stdout=output)
    )
    output.close()

In [17]:
for file in os.listdir('./pre-aln'): #create alignments of individual gene files
    align(file)

CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L1.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L2.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L3.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L4.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L5.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L6.fasta'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/mafft/7.245-with-extensions/bin/mafft', '--reorder', '--auto', './pre-aln/L10.fasta'], retur

In [18]:
# INDIVIDUAL ALIGNMENT COMPLETE. NOW, CONCATENATE ALIGNMENTS.
aln_extension = '.aln'
genomes = {}
for aln in os.listdir('./aln'):
    if not aln.endswith(aln_extension):
        continue
    alignment = AlignIO.read('./aln/' + aln, 'fasta')
    genomes[aln] = set()
    for entry in alignment:
        genome = entry.description
        if genome in genomes[aln]:
            print('\t**Error, duplicated genome in %s: %s' %(aln, genome))
            break
        genomes[aln].add(genome)

In [19]:
genome_union = set.union(*genomes.values())
missing_genes = {} #keeps track of # of missing marker genes in each genome
concatenation = {}
for genome in genome_union:
    missing_genes[genome] = 0
    concatenation[genome] = Align.SeqRecord(Align.Seq(''))
    concatenation[genome].name = genome
    concatenation[genome].id = genome
    concatenation[genome].description = genome

In [20]:
total_genes = 0.0
current_position = 1
partitions = open('partitions.txt', 'w')
for aln in os.listdir('./aln'):
    if not aln.endswith(aln_extension):
        continue
    print(aln.replace(aln_extension, ''))
    tmp_aln = AlignIO.read('./aln/' + aln, 'fasta')
    aln_length = tmp_aln.get_alignment_length() #expected alignment size
    total_genes += aln_length
    
    for entry in tmp_aln:
        if len(entry) != aln_length:
            print('\t**Error, block "%s" has a different length than the rest of the MSA: %s'\
                 %(entry.name, aln))
        genome = entry.description
        concatenation[genome] += deepcopy(entry.seq)
        
    partitions.write('LG, %s = %i-%i\n' %(aln.replace(aln_extension, ''),\
                                          current_position, current_position+aln_length-1))
    current_position += aln_length
    
    for genome in genome_union.difference(genomes[aln]):
        concatenation[genome] += Align.Seq( '-' * aln_length)
        missing_genes[genome] += aln_length
partitions.close()

L1.fasta
L2.fasta
L3.fasta
L4.fasta
L5.fasta
L6.fasta
L10.fasta
L13.fasta
L14.fasta
L15.fasta
L18.fasta
L22.fasta
L23.fasta
L24.fasta
L29.fasta
S2.fasta
S3.fasta
S4.fasta
S5.fasta
S7.fasta
S8.fasta
S9.fasta
S10.fasta
S11.fasta
S12.fasta
S13.fasta
S14.fasta
S15.fasta
S17.fasta
S19.fasta


In [21]:
AlignIO.write(Align.MultipleSeqAlignment(concatenation.values()), 'concatenated_alignment.aln', 'fasta')

1